In [ ]:
import pandas as pd
df = pd.read_parquet('../processed_data/with_gpt_results_2nd_round.parquet')

In [ ]:
df.info()

In [ ]:
df['new_theme'].value_counts()

In [ ]:
df['year'] = df['date'].apply(lambda x: x[:4])
df['year'] = pd.to_numeric(df['year'])
# Drop data of year 2014 and before
df = df[df['year']>=2015]
df['year'].value_counts().sort_index()

In [ ]:
for i in range(len(df)):
    assert max(df.iloc[i]['scores_2']) == df.iloc[i]['new_theme']

In [ ]:
max_score = []
for i in range(len(df)):
    arr = df.iloc[i]['scores_2']
    if arr.size > 0:
        index_max = np.argmax(arr) + 1
        max_score.append(index_max)    
    else:
        max_score.append(-1)
    
df['max_score_index'] = max_score

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
wrong_result = df[df['max_score_index'] != df['new_theme']]
wrong_result[['scores_2', 'new_theme']]

In [ ]:
wrong_result[wrong_result['new_theme'] > 0][['id', 'scores_2', 'new_theme']]

In [ ]:
df.loc[df['new_theme'] == 9, 'new_theme'] = 7

In [ ]:
print(df['max_score_index'].value_counts())
print(df['new_theme'].value_counts().sort_index())

In [ ]:
df.info()

In [ ]:
df['author'].value_counts()

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

test = df['new_theme'] == 2
id_list = df[test]['id']
good_id = id_list[id_list >2000]

for i in good_id:
    row = df[df['id'] == i]
    text = row['fulltext']
    print(text)

In [ ]:
df.iloc[2584]

In [ ]:
df[df['author'=='習遦點伎煙']]

In [ ]:
df[test]['author'].count()

In [ ]:
1317/5190

In [ ]:
df.to_parquet('../processed_data/with_gpt_results_2nd_round.parquet')

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

test = df['new_theme'] == 4
print(len(df[test]['fulltext']))
df[test]['fulltext']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.countplot(data = df, x = df['new_theme'], hue = df['new_theme'], palette='viridis', width=0.5, legend=False)

# Sentiment

In [ ]:
# sentiment description
df['gpt_sentiment'].describe()

In [ ]:
df.groupby('gpt_sentiment').size()

In [ ]:
sns.countplot(data = df, x = df['gpt_sentiment'], hue = df['gpt_sentiment'], palette='viridis', width=0.5, legend=False)

In [ ]:
1350+708

In [ ]:
(439+ 421+ 129)/5367

## Sentiment by Themes

In [ ]:
# sns.countplot(data = df, hue = df['gpt_sentiment'], x = df['new_theme'])

In [ ]:
# average sentiment by theme
df.groupby('new_theme')['gpt_sentiment'].mean()

In [ ]:
plt.figure(figsize=(8, 6))
sns.violinplot(x='new_theme', y='gpt_sentiment', data=df, color='purple', inner=None, width=1)
plt.title('Violin Plot of Values by Category')
plt.show()

In [ ]:
# df_good = df[df['new_theme']>0]
sentiment_by_theme = df_good.pivot_table(index='new_theme', columns='gpt_sentiment', aggfunc='size', fill_value=0)
sentiment_by_theme

In [ ]:
(351+326+57)/816

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()
topic_legend = ['T1: Market Entry', 'T2: Shipping Prices', 'T3: Platform Payment', 'T4: Fraud & Dispute', 'T5: Regulation', 'T6: Solidarity', 'T7: General']
# Creating countplots for each theme
for i in range(1, 8):
    theme = df[df['new_theme'] == i]
    sns.countplot(data=theme, x=theme['gpt_sentiment'], hue = theme['gpt_sentiment'], 
                  palette='viridis', width=0.5, legend=False, ax=axes[i-1])
    axes[i-1].set_title(f'Sentiment Scores of {topic_legend[i-1]}')
    axes[i-1].set_xlabel('Sentiment Scores')
    axes[i-1].set_ylabel('Counts')

# Hide the last subplot (8th grid)
axes[-1].axis('off')

plt.tight_layout()
plt.savefig('../Graphs/sentiment_topic.png')
plt.show()

## Sentiment Over Years

In [ ]:
df['year'].value_counts().sort_index()

In [ ]:
sentiment_by_year = df.groupby('year')['gpt_sentiment'].mean()
sentiment_by_year

In [ ]:
# Creating a line plot
plt.figure(figsize=(8, 6))
sns.lineplot(x='year', y='gpt_sentiment', data=df, estimator='mean', errorbar = ('ci', 95), marker='o')
plt.title('Average Sentiment Across Years')
plt.ylabel('Average Sentiment Scores')
plt.xlabel('Years')
plt.ylim(1, 7)
plt.show()

In [ ]:
?sns.lineplot

## Sentiment Across platforms

In [ ]:
df['keyword'].value_counts()

In [ ]:
df.groupby('keyword')['gpt_sentiment'].mean()

In [ ]:
df.groupby('keyword')['gpt_sentiment'].median()

In [ ]:
df.groupby('keyword')['gpt_sentiment'].size()

In [ ]:
translation = {'平台':'Platform',
               '满帮':'Manbang',
               '货拉拉':'Huolala',
               '货车帮':'Huochebang',
               '运满满':'Yunmanman'         
}
df['keyword'] = df['keyword'].map(translation)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
axes = axes.flatten()

list = df['keyword'].unique()

# Select a font that supports CJK characters
plt.rcParams['font.family'] = 'AppleGothic'

# Creating countplots for each theme
for i in range(0,5):
    platform = list[i]
    slice = df[df['keyword']==platform]
    sns.countplot(data=slice, x=slice['gpt_sentiment'], hue = slice['gpt_sentiment'], 
                  palette='viridis', width=0.5, legend=False, ax=axes[i])
    axes[i].set_title(f'Sentiment Scores of {platform}')
    axes[i].set_xlabel('Sentiment Scores')
    axes[i].set_ylabel('Counts')

# Hide the last subplot (8th grid)
axes[-1].axis('off')

plt.tight_layout()
plt.show()

# Theme Distribution

## Themes Across Years

In [ ]:
# Theme distribution across years
fig, axes = plt.subplots(3, 3, figsize=(20, 10))
axes = axes.flatten()
# Creating countplots for each theme
for i in range(1, 8):
    theme = df[df['new_theme'] == i]
    sns.countplot(data=theme, x=theme['year'], hue = theme['year'], 
                  palette='viridis', width=0.5, legend=False, ax=axes[i-1])
    axes[i-1].set_title(f'Yearly Distribution of Topic {i}')
    axes[i-1].set_xlabel('Year')
    axes[i-1].set_ylabel('Theme counts')

# Hide the last subplot (8th grid)
axes[-2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:

plt.figure(figsize=(10, 6))

for i in range(1, 8):
    theme_data = df[df['new_theme'] == i].groupby('year').size().reset_index(name='counts')
    sns.lineplot(data=theme_data, x='year', y='counts', marker='o', label=f'Theme {i}')

plt.title('Yearly Distribution of Themes')
plt.xlabel('Year')
plt.ylabel('Counts')
plt.legend(title='Themes')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Sample DataFrame creation (replace this with your actual DataFrame)
# df = pd.DataFrame({
#     'year': [2010, 2010, 2011, 2011, 2012, 2012] * 3,
#     'new_theme': [1, 2, 1, 2, 1, 2] * 3,
#     'count': [10, 15, 20, 25, 5, 30] * 3
# })

# Initialize the figure with a 3x3 grid of subplots
fig, axes = plt.subplots(3, 3, figsize=(20, 10))
axes = axes.flatten()

# Create a line plot for each theme
for i in range(1, 8):
    theme_data = df[df['new_theme'] == i].groupby('year').size().reset_index(name='counts')
    sns.lineplot(data=theme_data, x='year', y='counts', marker='o', ax=axes[i-1])
    axes[i-1].set_title(f'Yearly Distribution of Theme {i}')
    axes[i-1].set_xlabel('Year')
    axes[i-1].set_ylabel('Counts')

# Hide the unused subplots
for i in range(7, 9):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# Exclude rows when theme == -1
# df_good = df[df['new_theme']>0]
# df_2024_drop = df[df['year']<2024]
df_select = df[(df['new_theme']>0) & (df['year']<2024)]

# Pivot the data to get yearly counts for each theme
pivot_df = df_select.pivot_table(index='year', columns='new_theme', aggfunc='size', fill_value=0)

# Ensure all categories are represented (in case some years miss some categories)
all_categories = df_select['new_theme'].unique()
pivot_df = pivot_df.reindex(columns=all_categories, fill_value=0)
pivot_df = pivot_df.sort_index(axis=1)

# Now pivot_df has years as index and one column for each theme with counts as values

# Creating the stackplot
plt.figure(figsize=(10, 6))
labels = [f'Theme {i}' for i in range(1,8)]
colors = ['#E63946', '#F1FAEE', '#A8DADC', '#457B9D', '#1D3557', '#F4A261', '#2A9D8F']

plt.stackplot(pivot_df.index, pivot_df.T, colors=colors, labels=labels)

# Customize the plot
plt.title('Distribution of Topics Across Years')
plt.xlabel('Year')
plt.ylabel('Count')
plt.legend(topic_legend, loc='upper right')

plt.savefig('../Graphs/topic_year.png')
plt.show()


In [ ]:
pivot_df

In [ ]:
# yearly_count_themes = {}
# for i in range(1,8):
#     theme_i = df[df['new_theme']==i]
#     yearly_count = theme_i.groupby('year')['id'].count()
#     y_i = yearly_count.tolist()
#     yearly_count_themes[i] = y_i
# print(yearly_count_themes)

# x = sorted(df['year'].unique())
# y = pd.Dataframe(yearly_count_themes)
# # [value for value in yearly_count_themes.values()]
# labels = [f'theme{i}' for i in range(1,8)]
# # plt.stackplot(x, y, labels=labels)

## Themes Across Platforms

In [ ]:
# # Exclude rows when theme == -1
df_good = df[df['new_theme']>0]

# Pivot the data to get counts for each theme across platforms
theme_platform = df_good.pivot_table(index='keyword', columns='new_theme', aggfunc='size', fill_value=0)

# Defining the new order for the index
new_order = ['Platform', 'Manbang', 'Yunmanman', 'Huochebang', 'Huolala']

# Reordering the DataFrame based on the new index order
theme_platform = theme_platform.loc[new_order]

theme_platform

In [ ]:
row_percentage = theme_platform.div(theme_platform.sum(axis=1), axis=0) * 100
row_percentage

In [ ]:
# column_percentage = theme_platform.div(theme_platform.sum(axis=0), axis=1) * 100
# column_percentage

In [ ]:
# total_sum = theme_platform.sum().sum()
# cell_percentage = (theme_platform / total_sum) * 100
# cell_percentage

In [ ]:
import numpy as np

In [ ]:
topic_legend_2 = ['T1:\nMarket Entry', 'T2:\nShipping Prices', 'T3:\nPlatform Payment', 'T4:\nFraud & Dispute', 'T5:\nRegulation', 'T6:\nSolidarity', 'T7:\nGeneral']

plt.figure(figsize=(13, 7))
sns.heatmap(row_percentage, annot=True, cmap="YlGnBu")
plt.title('Percentages of Topics across Platforms')
plt.xlabel('Topics')
plt.ylabel('Platforms')
plt.xticks(ticks=np.arange(0, 7) + 0.5, labels = topic_legend_2)
plt.savefig('../processed_data/topic_platform.png')
plt.show()

In [ ]:
# Creating a pivot table
theme_keyword = df_good.groupby(['keyword', 'new_theme']).size().unstack(fill_value=0)

# Initialize a figure and axis
plt.figure(figsize=(10, 6))

# Bottom tracker for each keyword's bar
bottom = [0] * len(theme_keyword)

# For each new_theme, plot a bar stack
for theme in theme_keyword.columns:
    plt.bar(theme_keyword.index, theme_keyword[theme], bottom=bottom, label=theme)
    bottom = [sum(x) for x in zip(bottom, theme_keyword[theme])]

# Adding labels, title, and legend
plt.xlabel('Keyword')
plt.ylabel('Count')
plt.title('Stacked Bar Plot of New Themes per Keyword')
plt.legend(title='New Theme')

# Show the plot
plt.show()


In [ ]:
df['year'].value_counts().sort_index()